In [11]:
import tensorflow_datasets as tf
from tensorflow.keras.utils import to_categorical
import tensorflow as t;

In [6]:
(train_ds, train_labels), (test_ds, test_labels) = tf.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ASUS\tensorflow_datasets\tf_flowers\3.0.1.incompleteKR8UWA\tf_flowers-train.tfrecord*...:  …

Dataset tf_flowers downloaded and prepared to C:\Users\ASUS\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


In [12]:
train_ds = t.image.resize(train_ds, (150, 150))
test_ds = t.image.resize(test_ds, (150, 150))

NameError: name 'tf_flowers' is not defined

In [13]:
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [14]:

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [15]:
#load vgg16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

58889256/58889256 [==============================] - 6s 0us/step


In [16]:
#preprocessinpu
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [18]:
#to remove classification layer set it to false
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [19]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

In [20]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 213s 3s/step - loss: 1.8266 - accuracy: 0.3708 - val_loss: 1.3420 - val_accuracy: 0.4553
Epoch 2/50
65/65 [==============================] - 240s 4s/step - loss: 1.1683 - accuracy: 0.5367 - val_loss: 1.2365 - val_accuracy: 0.5175
Epoch 3/50
65/65 [==============================] - 75155s 1174s/step - loss: 0.9736 - accuracy: 0.6214 - val_loss: 1.2100 - val_accuracy: 0.6089
Epoch 4/50
30/65 [============>.................] - ETA: 1:10 - loss: 0.8090 - accuracy: 0.6969

KeyboardInterrupt: 